In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [48]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import feature_column

# Reference: https://www.tensorflow.org/tutorials/structured_data/feature_columns
# Used to parse csv data and convert to data sets 

# upload the heart_train.csv and heart_test.csv
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
train_df = pd.read_csv('heart_train.csv')

cols = []

for header in ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']:
  cols.append(tf.feature_column.numeric_column(header))

train_df['famhist'] = train_df['famhist'].apply(str)
famhist_train = tf.feature_column.categorical_column_with_vocabulary_list('famhist', ['Present', 'Absent'])
famhist_trainH = tf.feature_column.indicator_column(famhist_train)
cols.append(famhist_trainH)

# taken directly from: https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, batch_size=32):
  df = df.copy()
  labels = df.pop('chd')
  return tf.data.Dataset.from_tensor_slices((dict(df), labels)) \
          .shuffle(buffer_size=len(df)) \
          .batch(batch_size)


test_df = pd.read_csv('heart_test.csv')
test_df['famhist'] = test_df['famhist'].apply(str)
famhist_test = tf.feature_column.categorical_column_with_vocabulary_list('famhist', ['Present', 'Absent'])
famhist_testH = tf.feature_column.indicator_column(famhist_test)

train_dataset = df_to_dataset(train_df)
test_dataset = df_to_dataset(test_df)

#df.head()
# df2.head()
# for i in cols:
#   print(i)



In [0]:
#df2.head()

In [60]:
print("--Make model--")
model = tf.keras.models.Sequential([
  tf.keras.layers.DenseFeatures(cols),
  tf.keras.layers.Dense(256, 'relu'),
  tf.keras.layers.Dense(256, 'relu'),
  tf.keras.layers.Dense(256, 'relu'),
  tf.keras.layers.Dense(256, 'relu'),
  tf.keras.layers.Dense(units = 1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("--Fit model--")
model.fit(train_dataset, epochs= 300 , verbose=2)

print("--Evaluate model--")
model_loss, model_acc = model.evaluate(test_dataset, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuray: {model_acc*100:.1f}%")


--Make model--
--Fit model--

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/300
13/13 - 1s - loss: 1.9231 - accuracy: 0.5865
Epoch 2/300
13/13 - 0s - loss: 0.6997 - accuracy: 0.6082
Epoch 3/300
13/13 - 0s - loss: 0.6417 - accuracy: 0.6611
Epoch 4/300
13/13 - 0s - loss: 0.6266 - accuracy: 0.6562
Epoch 5/300
13/13 - 0s - loss: 0.6209 - accuracy: 0.6562
Epoch 6/300
13/13 - 0s - loss: 0.6194 - accuracy: 0.6346
Epoch 7/300
13/13 - 0s - loss: 0.5842 - accuracy: 0.6538
Epoch 8/300
13/13 - 0s - loss: 0.6590 - accuracy: 0.6154
Epoch 9/300
13/13 - 0s 